# Zero-Coupon Bond Using Marlowe Runtime\'s Command-Line Interface

The zero-coupon bond example is a simple Marlowe contract where a lender provides principal to a borrower who repays it back with interest.

In this demonsration we use Marlowe Runtime\'s command-line interface, `marlowe`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use addresses.

In [Marlowe Playground](https://play.marlowe-finance.io/), the contract looks like this in Blockly format.

![Zero-coupon bond Marlowe contract](images/01-zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Address "$LENDER_ADDR")
            (Address "$LENDER_ADDR")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Address "$LENDER_ADDR")
            (Party (Address "$BORROWER_ADDR"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Address "$BORROWER_ADDR")
                        (Address "$BORROWER_ADDR")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Address "$BORROWER_ADDR")
                        (Party (Address "$LENDER_ADDR"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close 
```

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.
- `CARDANO_RT_HOST`: IP address of the Marlowe Runtime proxy server.
- `CARDANO_RT_PORT`: Port number for the Marlowe Runtime proxy server.

It also assumes that the Lender and Borrower parties have addresses, signing keys, and funds.
- Lender
    - [keys/lender.address](keys/lender.address): Cardano address for the lender
    - [keys/lender.skey](keys/lender.skey): location of signing key file for the lender
- Borrower
    - [keys/borrower.address](keys/borrower.address): Cardano address for the borrower
    - [keys/borrower.skey](keys/borrower.skey): location of signing key file for the borrower

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [1]:
if [[ -z "$MARLOWE_RT_PORT" ]]
then

  if command -v podman > /dev/null
  then
    DOCKER_CLI=podman
  else
    DOCKER_CLI=docker
  fi
  
  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$($DOCKER_CLI volume inspect marlowe-compose_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for `marlowe`.
  export MARLOWE_RT_HOST="127.0.0.1"
  export MARLOWE_RT_PORT=3700

fi

# FIXME: This should be inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should be set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_HOST = $MARLOWE_RT_HOST"
echo "MARLOWE_RT_PORT = $MARLOWE_RT_PORT"

CARDANO_NODE_SOCKET_PATH = /home/bbush/.local/share/containers/storage/volumes/marlowe-compose_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1
MARLOWE_RT_HOST = 127.0.0.1
MARLOWE_RT_PORT = 3700


### Lender address and funds

Check that an address and key has been created for the lender. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [2]:
LENDER_SKEY=keys/lender.skey
LENDER_ADDR=$(cat keys/lender.address)
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


Check that the lender has at least one hundred ada.

In [3]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
78690d72c67960d4600889d7dd7da2069f14658ed6791c8838dccdd33fe3971f     1        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [4]:
echo "$EXPLORER_URL/address/$LENDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Borrower address and funds

Check that an address and key has been created for the borrower. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [5]:
BORROWER_SKEY=keys/borrower.skey
BORROWER_ADDR=$(cat keys/borrower.address)
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


Check that the borrower has at least one hundred ada.

In [6]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
78690d72c67960d4600889d7dd7da2069f14658ed6791c8838dccdd33fe3971f     2        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [7]:
echo "$EXPLORER_URL/address/$BORROWER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


## Design the contract

The zero-coupon bond contract can be downloaded from the [Marlowe Playground](https://play.marlowe-finance.io/) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Here we generate the contract using Marlowe CLI.

First, set the loan's principal to 80 ada and its interest to 5 ada.

In [8]:
ADA=1000000  # 1 ada = 1,000,000 lovelace

PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))

echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

PRINCIPAL = 80000000 lovelace
INTEREST = 5000000 lovelace


On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [9]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Next find the current time, measured in [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [10]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

NOW="$(($(date -u +%s) * SECOND))"
echo "NOW = $NOW POSIX milliseconds = $(date -d @$((NOW / SECOND)))"

NOW = 1678902170000 POSIX milliseconds = Wed Mar 15 11:42:50 AM MDT 2023


The contract has a lending deadline and a repayment deadline. For convenience in this example, set the deadlines to the near future.

In [11]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo "LENDER_DEADLINE = $LENDER_DEADLINE POSIX milliseconds = $(date -d @$((LENDER_DEADLINE / SECOND)))"
echo "BORROWER_DEADLINE = $BORROWER_DEADLINE POSIX milliseconds = $(date -d @$((BORROWER_DEADLINE / SECOND)))"

LENDER_DEADLINE = 1678905770000 POSIX milliseconds = Wed Mar 15 12:42:50 PM MDT 2023
BORROWER_DEADLINE = 1678912970000 POSIX milliseconds = Wed Mar 15 02:42:50 PM MDT 2023


Now create the JSON file for the contract, `zcb-contract.json`.

In [12]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender "$LENDER_ADDR" \
  --borrower "$BORROWER_ADDR" \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file zcb-state.json

The various command-line options are described by the help system.

In [13]:
marlowe-cli template zcb --help

Usage: marlowe-cli template zcb --minimum-ada INTEGER --lender PARTY
                                --borrower PARTY --principal INTEGER
                                --interest INTEGER --lending-deadline TIMEOUT
                                --repayment-deadline TIMEOUT

  Create a zero-coupon bond.

Available options:
  --minimum-ada INTEGER    Lovelace that the lender contributes to the initial
                           state.
  --lender PARTY           The lender.
  --borrower PARTY         The borrower.
  --principal INTEGER      The principal, in lovelace.
  --interest INTEGER       The interest, in lovelace.
  --lending-deadline TIMEOUT
                           The lending deadline. POSIX milliseconds or duration:
                           `INTEGER[s|m|d|w|h]`.
  --repayment-deadline TIMEOUT
                           The repayment deadline. POSIX milliseconds or
                           duration: `INTEGER[s|m|d|w|h]`.
  -h,--help                Show this help text


## Examine the contract

View the contract file as YAML.

In [14]:
json2yaml zcb-contract.json

timeout: 1678905770000
timeout_continuation: close
when:
- case:
    deposits: 80000000
    into_account:
      address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
  then:
    from_account:
      address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
    pay: 80000000
    then:
      timeout: 1678912970000
      timeout_continuation: close
      when:
      - case:
          deposits:
            add: 80000000
            and: 5000000
          into_account:
            address: addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
        then:
          from_account:
            address: addr_test1vpy4n4peh4su

### \[Optional\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe-finance.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe-finance.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

Here we\'ll perform step 6. First we bundle the contract and its initial state into a single file.

In [15]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file zcb-contract.json \
  --state-file zcb-state.json \
  --out-file zcb-marlowe.json

Now we analyze the contract and its execution paths.

In [16]:
marlowe-cli run analyze \
  --marlowe-file zcb-marlowe.json

Note that path-based analysis ignore the initial state of the contract and instead start with an empty state.
Starting search for execution paths . . .
 . . . found 3 execution paths.
- Preconditions:
    Duplicate accounts: []
    Duplicate bound values: []
    Duplicate choices: []
    Invalid account parties: []
    Invalid account tokens: []
    Invalid choice parties: []
    Invalid roles currency: false
    Non-positive account balances: []
- Role names:
    Blank role names: false
    Invalid role names: []
- Tokens:
    Invalid tokens: []
- Maximum value:
    Actual: 88
    Invalid: false
    Maximum: 5000
    Percentage: 1.76
    Unit: byte
- Minimum UTxO:
    Requirement:
      lovelace: 1120600
- Execution cost:
    Memory:
      Actual: 6588250
      Invalid: false
      Maximum: 14000000
      Percentage: 47.058928571428574
    Steps:
      Actual: 1767311958
      Invalid: false
      Maximum: 10000000000
      Percentage: 17.67311958
- Transaction size:
    Actual: 1630


In the above report, we see that the contract doesn\'t have any duplicate or invalid values, and it does not exceed any of the blockchain\'s protocol parameters. In particular, note that our previously chosen `MIN_LOVELACE` value of 2 ada is greater than the 1.120600 ada that the analysis tool says is needed. Thus, it is safe to execute any path in the contract.

## Transaction 1. Create the Contract

Marlowe Runtime\'s command `marlowe create` will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [17]:
marlowe create --help

Usage: marlowe create --change-address ADDRESS [-a|--address ADDRESS] 
                      [--collateral-utxo UTXO] --manual-sign FILE_PATH 
                      [-m|--metadata-file FILE_PATH] [--tags-file FILE_PATH] 
                      [--v1] 
                      [(-r|--role ROLE=ADDRESS) | 
                        --roles-config-file FILE_PATH | 
                        --role-token-policy-id POLICY_ID] 
                      (--core-file FILE_PATH | --contract-file FILE_PATH 
                        [--args-file FILE_PATH | 
                          [--timeout-arg NAME=POSIX_TIMESTAMP] 
                          [--value-arg NAME=INTEGER]]) --min-utxo LOVELACE

  Create a new Marlowe Contract

Available options:
  --change-address ADDRESS The address to which the change of the transaction
                           should be sent.
  -a,--address ADDRESS     An address whose UTXOs can be used as inputs to the
                           transaction
  --collateral-utxo UTXO   

In [18]:
CONTRACT_ID=$(
marlowe create \
  --core-file zcb-contract.json \
  --min-utxo "$MIN_LOVELACE" \
  --change-address "$LENDER_ADDR" \
  --manual-sign tx-1.unsigned \
| jq -r 'fromjson | .contractId' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952#1


Marlowe Runtime uses the first (creation) UTxO of the contract to identify it throughout its lifecycle.

The result of building the transaction is the identifier for the contract and the file `tx-1.unsigned`, which contains the Cardano unsigned transaction for creating the contract, in text-envelope format.

In [19]:
json2yaml tx-1.unsigned

cborHex: 86a4008182582078690d72c67960d4600889d7dd7da2069f14658ed6791c8838dccdd33fe3971f010182a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b793fffa300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e848002820058206328e20e31ca15459cd77edcabaaa25b6ae91e31bdc99c709aca77b77cc5fe30021a000305810b58208a0a2a9096cdece9c5ebe0a09489bdcf3d929459ab07c58d56d902ab41a44a749fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c1b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581c1b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59ffd87a80ffffd8799fd87980d8799fd8799f581c1b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59ffd87a80ffffd8799f4040ffd87a9f1a04c4b400ffffd87a9fd8799fd87980d8799fd8799f581c1b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59ffd87a80ffffd87a9fd8799fd87980d8799fd8799f581c4959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [20]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [21]:
echo "$EXPLORER_URL/transaction/$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952?tab=utxo


One can also examine the contract's UTxO using `cardano-cli`.

In [22]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_1#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "6328e20e31ca15459cd77edcabaaa25b6ae91e31bdc99c709aca77b77cc5fe30"


## View the details of the contract on the blockchain

Marlowe Runtime\'s command `marlowe log` can fetch a contract from the blockchain and print information about it.

In [23]:
marlowe log --show-contract "$CONTRACT_ID"

transaction b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952 (creation)
ContractId:      b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952#1
SlotNo:          23219009
BlockNo:         728217
BlockId:         da20a13cbb6005d39b728a7db51aa732559447d62bcd3937fffa5693e0c1cb23
ScriptAddress:   addr_test1wqhdyccahvnheppng3fut3phhp3jt5m37zp4529ezz535ms2u9jqv
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck") (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck")
            (Token "" "")
            (Constant 80000000))
         (Pay (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck")
            (Party (Address "addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d"))
            (Token "" "")
            (Constant 80000000)
            (When [
               (Case
                  (Deposit (Address "addr_test1vpy4n4peh4s

More detail can be retrieved using `marlowe-pipe`.

In [24]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

creation:
  output:
    address: 702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e
    assets:
      ada: 2000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1678905770000
        timeout_continuation: close
        when:
        - case:
            deposits: 80000000
            into_account:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
          then:
            from_account:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
            pay: 80000000
            then:
              timeout: 1678912970000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 80000000
                    and: 5000000
     

## Transaction 2. The lender deposits the principal

The lender deposits their 80 ada of principal into the contract using Marlowe Runtime\'s `marlowe deposit` command. The lender is providing the funding for and receiving the change from this transaction, so we provide their address. We provide the contract identifier and save the unsigned transaction in the file `tx-2.unsigned`.

In [25]:
marlowe deposit --help

Usage: marlowe deposit --change-address ADDRESS [-a|--address ADDRESS] 
                       [--collateral-utxo UTXO] --manual-sign FILE_PATH 
                       [-m|--metadata-file FILE_PATH] [--tags-file FILE_PATH]
                       (-c|--contract CONTRACT_ID) --to-party ROLE_NAME|ADDRESS
                       --from-party ROLE_NAME|ADDRESS 
                       ((-c|--currency MINTING_POLICY_ID)
                         (-n|--token-name TOKEN_NAME) (-q|--quantity INTEGER) |
                         (-l|--lovelace INTEGER)) 
                       [--continuation-file FILE_PATH] 
                       [-l|--validity-lower-bound TIMESTAMP] 
                       [-u|--validity-upper-bound TIMESTAMP]

  Deposit funds into a contract

Available options:
  --change-address ADDRESS The address to which the change of the transaction
                           should be sent.
  -a,--address ADDRESS     An address whose UTXOs can be used as inputs to the
                     

In [26]:
TX_2=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$LENDER_ADDR" \
  --lovelace "$PRINCIPAL" \
  --change-address "$LENDER_ADDR" \
  --manual-sign tx-2.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_2 = $TX_2"

TX_2 = 49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202


Note that if the transaction would violate the logic of the Marlowe contract, one would receive an error message. For example, let\'s say that we deposit the incorrect amount or deposit it to the wrong party\'s internal account.

In [27]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$LENDER_ADDR" \
  --lovelace 80 \
  --change-address "$LENDER_ADDR" \
  --manual-sign /dev/null

ApplyFailed (ApplyInputsConstraintsBuildupFailed (MarloweComputeTransactionFailed "TEApplyNoMatchError"))


: 1

In [28]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$BORROWER_ADDR" \
  --lovelace "$PRINCIPAL"\
  --change-address "$LENDER_ADDR" \
  --manual-sign /dev/null

ApplyFailed (ApplyInputsConstraintsBuildupFailed (MarloweComputeTransactionFailed "TEApplyNoMatchError"))


: 1

The [Marlowe Debugging Cookbook](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/debugging-cookbook.md) guides interpretation of error messages. Also, one can determine the possible actions for the contract at its current stage of execution by studing the contract\'s current state or by using Marlowe playground to simulate the contract.

![Simulation of zero-coupon bond contract in Marlowe Playground](images/zcb-simulation.png)

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [29]:
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600

TxId "49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202"


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [30]:
echo "$EXPLORER_URL/transaction/$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202?tab=utxo


One can see that the lender has 82 ada less than originally. Two ada were deposited in the contract when it was created and 80 ada were paid to the borrower in the second transaction.

In [31]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202     0        917043702 lovelace + TxOutDatumNone


The borrower has an additional 80 ada (the loan's principal) now.

In [32]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202     2        80000000 lovelace + TxOutDatumNone
78690d72c67960d4600889d7dd7da2069f14658ed6791c8838dccdd33fe3971f     2        1000000000 lovelace + TxOutDatumNone


The Marlowe contract still has the 2 ada from its creation.

In [33]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "e31176d1805035fa5d2ead40ea09a9f672926d29081d3ec2764372b11c06c34f"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s command `marlowe log` can fetch a contract from the blockchain and print information about it.

In [34]:
marlowe log --show-contract "$CONTRACT_ID"

transaction b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952 (creation)
ContractId:      b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952#1
SlotNo:          23219009
BlockNo:         728217
BlockId:         da20a13cbb6005d39b728a7db51aa732559447d62bcd3937fffa5693e0c1cb23
ScriptAddress:   addr_test1wqhdyccahvnheppng3fut3phhp3jt5m37zp4529ezz535ms2u9jqv
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck") (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck")
            (Token "" "")
            (Constant 80000000))
         (Pay (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck")
            (Party (Address "addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d"))
            (Token "" "")
            (Constant 80000000)
            (When [
               (Case
                  (Deposit (Address "addr_test1vpy4n4peh4s

More detail can be retrieved using `marlowe-pipe`.

In [35]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

creation:
  output:
    address: 702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e
    assets:
      ada: 2000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1678905770000
        timeout_continuation: close
        when:
        - case:
            deposits: 80000000
            into_account:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
          then:
            from_account:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
            pay: 80000000
            then:
              timeout: 1678912970000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 80000000
                    and: 5000000
     

## Transaction 3. The borrower repays the loan

After some time passes, the borrower repays principal plus interest. Thus, they fund the transaction and receive the change at their address.

In [36]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$BORROWER_ADDR" \
  --to-party "$BORROWER_ADDR" \
  --lovelace "$((PRINCIPAL+INTEREST))" \
  --change-address "$BORROWER_ADDR" \
  --manual-sign tx-3.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

TX_3 = 16c757192c0e32bed9a89d48fdf7705f5557430ee8bc6b4ee9e122f57519e286


Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [37]:
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600

TxId "16c757192c0e32bed9a89d48fdf7705f5557430ee8bc6b4ee9e122f57519e286"


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [38]:
echo "$EXPLORER_URL/transaction/$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/16c757192c0e32bed9a89d48fdf7705f5557430ee8bc6b4ee9e122f57519e286?tab=utxo


One can see that the lender received back the 80 ada of principal and the 2 ada deposited when the contract was created, along with the additional 5 ada of interest, totallying 87 ada.

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
16c757192c0e32bed9a89d48fdf7705f5557430ee8bc6b4ee9e122f57519e286     1        87000000 lovelace + TxOutDatumNone
49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202     0        917043702 lovelace + TxOutDatumNone


The borrower now has about 5 ada (the loan's interest) less than originally.

In [40]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
16c757192c0e32bed9a89d48fdf7705f5557430ee8bc6b4ee9e122f57519e286     0        914430494 lovelace + TxOutDatumNone
49d47a1daccd37f21260c4c5da2242a3a3ca571ef9db38e8760a1597f4b08202     2        80000000 lovelace + TxOutDatumNone


The Marlowe contract has closed, so there is no output to its script address.

## View the completion of the contract on the blockchain

Marlowe Runtime\'s command `marlowe log` can fetch a contract from the blockchain and print information about it.

In [41]:
marlowe log --show-contract "$CONTRACT_ID"

transaction b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952 (creation)
ContractId:      b7797f9b7bc63361a30f84bb61ea9759bc0bbf105ceb4adcb08fef63b1e8d952#1
SlotNo:          23219009
BlockNo:         728217
BlockId:         da20a13cbb6005d39b728a7db51aa732559447d62bcd3937fffa5693e0c1cb23
ScriptAddress:   addr_test1wqhdyccahvnheppng3fut3phhp3jt5m37zp4529ezz535ms2u9jqv
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck") (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck")
            (Token "" "")
            (Constant 80000000))
         (Pay (Address "addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck")
            (Party (Address "addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d"))
            (Token "" "")
            (Constant 80000000)
            (When [
               (Case
                  (Deposit (Address "addr_test1vpy4n4peh4s

More detail can be retrieved using `marlowe-pipe`.

In [42]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

creation:
  output:
    address: 702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e
    assets:
      ada: 2000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1678905770000
        timeout_continuation: close
        when:
        - case:
            deposits: 80000000
            into_account:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
          then:
            from_account:
              address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
            pay: 80000000
            then:
              timeout: 1678912970000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 80000000
                    and: 5000000
     